#### @prompt example

In [3]:
from openbb import obb
import os
from magentic import prompt
from openbb import obb
obb.account.login(pat=os.environ['PAT_KEY'])
FMP_KEY = os.environ['FMP_KEY']


## Six Step Dance via Langchain
1. Taking into Account Company Industry (Growth / Value)
    1. Growth / Value
    2. Sector outperforming market or not
    3. Company outperforming sector
    4. PE vs Industry
2. Determining Company Financial Stability
    1. Debt To Equity
    2. Current Ratio
    3. Interest Margin Cover
3. Looking at Historical Earning and Growth
    1. Sales Growth
    2. EPS Growth
    3. Return on Invested Capital
    4. Profit Margin
    5. Cashflow Growth
    6. Equity Growth
    7. Return on Equity
4. Understanding Earnings and Sales Expectations
    1. Analyst Estimates
    2. PEG
5. Checking out competition
    1. P/E
    2. P/S
    3.MOAT
6. Estimating Company Values
    1. Insider Buying
    2. Senate TRading
    3. 13f

## Buffett Criteria
### Gross Margin Indicates profitability > 40% GrossProfit/Revenue GROSS PM close to 40 for recent years
### SG&A Margin efficiency in managing overhead costs < 30% SG&A Expenses / Gross Profit
### R&D Margin r&d spending relative to profitability < 30% R&D Expenses / Gross Profit
### Interest Expense Margin reliance on debt and financial health < 15 % Interest Expense / Operating Income
### Income Tax Margin Gauges tax efficiency ~ 20% Income Tax / Pre-Tax Income
### Profit Margin Shows overall profitabilty relative to revenue > 20% Net Income / Revenue
### EPS Growth Indicates growth and consistency in earnings >0 and growing Net Income / Shares Outstanding




In [4]:
obb.news.world(provider='fmp').to_llm()

'[{"date":"2025-03-22T16:01:06","title":"Ways to Leverage Inverse ETFs, Commodities Until \\"Overdue\\" Market Rally","images":[{"o":"https:\\/\\/images.financialmodelingprep.com\\/news\\/ways-to-leverage-inverse-etfs-commodities-until-overdue-market-20250322.jpg"}],"text":"Chris Vermeulen is bearish on mega and small caps until there\'s clear indicators the markets will rally off current levels. He urges investors to look into inverse ETFs as a way to play on bearish action.","url":"https:\\/\\/www.youtube.com\\/watch?v=vmpSy8uYgYI","site":"youtube.com"},{"date":"2025-03-22T14:05:29","title":"Will \'Drill, Baby, Drill\' Work for the US Economy?","images":[{"o":"https:\\/\\/images.financialmodelingprep.com\\/news\\/will-drill-baby-drill-work-for-the-us-economy-20250322.jpg"}],"text":"Bloomberg\'s Alix Steel talks to oil producers and policymakers at CERA week. Saad Rahim of Trafigura and Harold Hamm of Continental Resources tell us why an industry that wanted to pump more oil into the 

In [5]:



@prompt("Explain like I'm five this financial concept: {concept}")
def explain(concept: str) -> str: ...


explain("Subprime mortgage crisis")

"Okay! Imagine you have a toy store, and you want to lend your toys to friends. You have friends who always return your toys on time and friends who sometimes forget. Usually, you lend your toys to the friends who will return them because you trust them more.\n\nNow, let's say someone told you it's okay to lend toys to the friends who might forget because you have lots of toys, and you thought those friends would start remembering to return them. But then, many of those friends forgot to return the toys, and suddenly you didn't have enough toys left to play with or lend to others.\n\nThe subprime mortgage crisis is kind of like that, but with money. Banks lent money to people who might have had trouble paying it back (like the forgetful friends), thinking it would be okay. But then, lots of people couldn't pay back the money, and it caused big problems for the banks and the whole economy, like a broken toy store."

In [6]:

from magentic import prompt
from pydantic import BaseModel


class Portfolio(BaseModel):
    equity_etf_pct: float
    bond_etf_pc: float
    crypto_etf_pc: float
    commodities_pc: float
    reasoning: str


@prompt("Create a strong portfolio of {size} allocation size.")
def create_portfolio(size: str) -> Portfolio: ...


portfolio = create_portfolio("$50,000")

#### @chatprompt decorator

In [7]:
from magentic import chatprompt, AssistantMessage, SystemMessage, UserMessage
from pydantic import BaseModel

class Quote(BaseModel):
    quote: str
    person: str


@chatprompt(
    SystemMessage("You are an avid reader of financial literature."),
    UserMessage("What is your favorite quote from Warren Buffet?"),
    AssistantMessage(
        Quote(
            quote="Price is what you pay; value is what you get.",
            person="Warren Buffet",
        )
    ),
    UserMessage("What is your favorite quote from {person}?"),
)
def get_finance_quote(person: str) -> Quote: ...


get_finance_quote("Charlie Munger")

Quote(quote='The big money is not in the buying or the selling, but in the waiting.', person='Charlie Munger')

#### Function calling

In [8]:
import os
import requests
from magentic import prompt, FunctionCall
import logging
from openbb import obb
from langchain.tools import Tool
import reticker


def get_income_statement(query:str) -> str:
    try:
        ticker = reticker(query)
        period = 'annual' if 'annual' in query else 'quarterly'
        return obb.equity.fundamental.income(symbol=ticker, period=period, limit=5, provider='fmp').to_llm()
    except Exception as e:
        return f'Unable to find data for {ticker}'

def get_balance_sheet(query:str) -> str:
    try:
        ticker = reticker(query)
        period = 'annual' if 'annual' in query else 'quarterly'
        return obb.equity.fundamental.balance(symbol=ticker, period=period, limit=5, provider='intrinio')
    except Exception as e:
        return f'Unable to find data for {ticker}'


def get_aggressive_smallcaps() -> list :
    return obb.equity.discovery.aggressive_small_caps(sort='desc').to_llm()


def get_undervalued_growth() -> list:
    return obb.equity.discovery.undervalued_growth(provider='yfinance').to_llm()


def get_ticker_from_query(query):
    extractor = reticker.TickerExtractor(deduplicate=True)
    tickers = extractor.extract(query)
    if len(tickers) > 1:
        return ','.join(tickers)
    return tickers[0] 


def get_stock_price(query: str) -> str:
    """Get the current stock price for a given ticker."""
    try:
        ticker = get_ticker_from_query(query)
        data = obb.equity.price.quote(symbol=ticker)

        return data.to_llm()
    except Exception as e:
        return f"Error fetching stock price for {ticker}: {str(e)}"

def get_company_overview(query: str) -> str:
    """Get an overview of a company for a given ticker."""
    try:
        ticker = get_ticker_from_query(query)
        data = obb.equity.profile(symbol=ticker)
        return data.to_llm()
    except Exception as e:
        return f"Error fetching company overview for {ticker}: {str(e)}"
    
def get_group_performance(group:str) -> list:
    """ Get performane by industry or sector"""
    data = obb.equity.compare.groups(group=group, metric='performance', provider='finviz')
    return data.to_llm()

def get_latest_news_for_company(query : str) -> str:
    """ Get latest news for a company """
    try:
        ticker = get_ticker_from_query(query)
        return obb.news.company(symbol=ticker)
    except Exception as e:
        return f"Error fetching stock price for {ticker}: {str(e)}"

@prompt(
    "Use the appropriate search function to answer: {question}",
    functions=[get_stock_price, get_aggressive_smallcaps, get_undervalued_growth, get_latest_news_for_company,
               get_balance_sheet, get_income_statement, get_group_performance],
)
def perform_search(question: str) -> FunctionCall[str]: ...


output = perform_search("Find the best performing industry")
result = output()
from pprint import pprint
pprint(result)

('[{"name":"Coking '
 'Coal","performance_1d":-0.0089,"performance_1w":-0.0496,"performance_1m":-0.1058,"performance_3m":-0.2851,"performance_6m":-0.1882,"performance_1y":-0.4023,"performance_ytd":-0.2323,"analyst_recommendation":2.01,"volume":19280000,"volume_average":3840000,"volume_relative":5.02},{"name":"Business '
 'Equipment & '
 'Supplies","performance_1d":-0.0333,"performance_1w":-0.0435,"performance_1m":-0.1473,"performance_3m":-0.1986,"performance_6m":-0.176,"performance_1y":-0.1542,"performance_ytd":-0.1588,"analyst_recommendation":1.0,"volume":4720000,"volume_average":1420000,"volume_relative":3.32},{"name":"Textile '
 'Manufacturing","performance_1d":-0.0331,"performance_1w":-0.0404,"performance_1m":-0.1383,"performance_3m":-0.1149,"performance_6m":-0.1907,"performance_1y":-0.219,"performance_ytd":-0.119,"analyst_recommendation":2.01,"volume":926890,"volume_average":305300,"volume_relative":3.04},{"name":"Steel","performance_1d":-0.031,"performance_1w":-0.0287,"performanc

#### Prompt Chains

In [9]:
import csv
from magentic import prompt_chain


def get_earnings_calendar(ticker: str, api_key: str = FMP_KEY) -> list:
    url = f"https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&symbol={ticker}&horizon=12month&apikey={api_key}"
    with requests.Session() as s:
        download = s.get(url)
        decoded_content = download.content.decode("utf-8")
        cr = csv.reader(decoded_content.splitlines(), delimiter=",")
        my_list = list(cr)
    return my_list


@prompt_chain(
    "What's {ticker} expected earnings dates for the next 12 months?",
    functions=[get_earnings_calendar],
)
def get_earnings(ticker: str) -> str: ...


get_earnings("IBM")

'The expected earnings dates for IBM in the next 12 months are:\n\n1. April 22, 2025 - Reporting for the fiscal quarter ending March 31, 2025.\n2. July 22, 2025 - Reporting for the fiscal quarter ending June 30, 2025.'

#### Streaming Response

In [ ]:
from magentic import StreamedStr


@prompt("Explain to me {term} in a way a 5-year-old would understand.")
def describe_finance_term(term: str) -> StreamedStr: ...


# Print the chunks while they are being received
for chunk in describe_finance_term("liquidity"):
    print(chunk, end="")

#### Streaming Structured Outputs

In [ ]:
from collections.abc import Iterable
from time import time


class Portfolio(BaseModel):
    equity_etf_pct: float
    bond_etf_pc: float
    crypto_etf_pc: float
    commodities_pc: float
    reasoning: str


@prompt("Create {n_portfolio} portfolios with varying deegress of risk apetite.")
def create_portfolios(n_portfolio: int) -> Iterable[Portfolio]: ...


start_time = time()
for portfolio in create_portfolios(3):
    print(f"{time() - start_time:.2f}s : {portfolio}")

#### Asynchronous Streaming

In [ ]:
import asyncio
from typing import AsyncIterable


@prompt("List three high-growth stocks.")
async def iter_growth_stocks() -> AsyncIterable[str]: ...


@prompt("Tell me more about {stock_symbol}")
async def tell_me_more_about(stock_symbol: str) -> str: ...


start_time = time()
tasks = []
async for stock in await iter_growth_stocks():
    # Use asyncio.create_task to schedule the coroutine for execution before awaiting it
    # This way descriptions will start being generated while the list of stocks is still being generated
    task = asyncio.create_task(tell_me_more_about(stock))
    tasks.append(task)

descriptions = await asyncio.gather(*tasks)

for desc in descriptions:
    print(desc)

### Attempting to write a Small Agent

In [15]:
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [ ]:
@tool
def get_performance(group:str) -> list:
    """ Return performance by sector or industyr industry performance for last week, last month, last quarter, last half year and last year"""
    return obb.equity.compare.groups(group=group, metric='performance', provider='finviz').to_llm()

@tool
def get_strong_buy_for_sector(sector : str) -> list :
    """ Return the strong buy recommendation for a given sector"""
    new_sector = '_'.join(sector.lower().split())
    data = obb.equity.screener(provider='finviz', sector=new_sector, recommendation='buy')
    return data.to_llm()

@tool
def get_best_stock_performers_for_sector(sector:str) -> list :
    """ Return the best  10 stock performers for last week and last month for a sector"""
    data = obb.equity.screener(provider='finviz', filters_dict={'Sector' : sector, 'Performance' : 'Week Up', 'Performance 2' : 'Month Up'}, limit=10)
    return data.to_llm()

@tool
def get_best_stock_performers_for_industry(industry:str) -> list :
    """ Return the best  10 stock performers for last week and last month for a sector"""
    data = obb.equity.screener(provider='finviz', filters_dict={'Sector' : industry, 'Performance' : 'Week Up', 'Performance 2' : 'Month Up'}, limit=10)
    return data.to_llm()

@tool
def get_valuation_for_group(group:str) -> list:
    """ Return P/E ratio for a group or industry"""
    return obb.equity.compare.groups(group=group, metric='valuation', provider='finviz').to_llm()

@tool
def get_valuation_for_company(ticker:str) -> list:
    """ Return Valuation Ratios  for a company"""#
    # requires obb login
    obb.account.login(pat=os.environ['PAT_KEY'])
    return obb.equity.fundamental.ratios_ttm(symbol='AAPL', provider='fmp', limit=1).to_llm()

#get_strong_buy_for_sector.invoke('Consumer Cyclical')
get_best_stock_performers_for_sector('Technology')

'[{"symbol":"VVPR","name":"VivoPower International PLC","country":"United Kingdom","sector":"Technology","industry":"Solar","market_cap":5730000.0,"price":1.29,"change_percent":0.5214,"volume":64658303},{"symbol":"OUST","name":"Ouster Inc","country":"USA","sector":"Technology","industry":"Electronic Components","market_cap":549970000.0,"price":11.05,"change_percent":0.3186,"volume":7915374},{"symbol":"AUID","name":"authID Inc","country":"USA","sector":"Technology","industry":"Software - Infrastructure","market_cap":80600000.0,"price":7.38,"change_percent":0.1752,"volume":263796},{"symbol":"TDTH","name":"Trident Digital Tech Holdings Ltd. ADR","country":"Singapore","sector":"Technology","industry":"Information Technology Services","market_cap":77770000.0,"price":1.38,"change_percent":0.1665,"volume":364280},{"symbol":"JNVR","name":"Janover Inc","country":"USA","sector":"Technology","industry":"Software - Infrastructure","market_cap":6950000.0,"price":4.93,"change_percent":0.1359,"volume

In [17]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful stock financial analys, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [ ]:
tools = [get_performance, get_strong_buy_for_sector, get_best_stock_performers_for_sector]
llm_with_tools = llm.bind_tools(tools)

In [19]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [20]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [21]:
list(agent_executor.stream({"input": "Find the performance of all sectors"}))



> Entering new None chain...

Invoking: `get_performance` with `{'group': 'sector'}`


[{"name":"Real Estate","performance_1d":-0.0111,"performance_1w":-0.0027,"performance_1m":-0.0267,"performance_3m":-0.0266,"performance_6m":-0.0765,"performance_1y":0.0618,"performance_ytd":0.0092,"analyst_recommendation":1.98,"volume":660630000,"volume_average":371640000,"volume_relative":1.78},{"name":"Consumer Defensive","performance_1d":-0.0018,"performance_1w":-0.0007,"performance_1m":-0.0333,"performance_3m":-0.026,"performance_6m":-0.0213,"performance_1y":0.0677,"performance_ytd":0.0093,"analyst_recommendation":2.01,"volume":907130000,"volume_average":462350000,"volume_relative":1.96},{"name":"Consumer Cyclical","performance_1d":0.0021,"performance_1w":0.0004,"performance_1m":-0.1137,"performance_3m":-0.1455,"performance_6m":0.0339,"performance_1y":0.0555,"performance_ytd":-0.095,"analyst_recommendation":1.79,"volume":2750000000,"volume_average":1420000000,"volume_relative":1.94},{"name":"Co

[{'actions': [ToolAgentAction(tool='get_performance', tool_input={'group': 'sector'}, log="\nInvoking: `get_performance` with `{'group': 'sector'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_G3t2uZBGzhoBhuUPI3kdJVss', 'function': {'arguments': '{"group":"sector"}', 'name': 'get_performance'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-018d44b8-aa43-48dc-a72f-77f0daca4311', tool_calls=[{'name': 'get_performance', 'args': {'group': 'sector'}, 'id': 'call_G3t2uZBGzhoBhuUPI3kdJVss', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'get_performance', 'args': '{"group":"sector"}', 'id': 'call_G3t2uZBGzhoBhuUPI3kdJVss', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_G3t2uZBGzhoBhuUPI3kdJVss')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_G3t2uZBGzhoBhuUPI3kdJVss', 'function': {'argume

### Chat Memory

In [29]:
from langchain_core.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful stock recommendation assistant , but dont know current events.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [30]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

In [31]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [34]:
input1 = "Which sector shown a constant postive performance across quarter, month and week?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `get_performance` with `{'group': 'sector'}`


[{"name":"Real Estate","performance_1d":-0.0111,"performance_1w":-0.0027,"performance_1m":-0.0267,"performance_3m":-0.0266,"performance_6m":-0.0765,"performance_1y":0.0618,"performance_ytd":0.0092,"analyst_recommendation":1.98,"volume":660630000,"volume_average":371640000,"volume_relative":1.78},{"name":"Consumer Defensive","performance_1d":-0.0018,"performance_1w":-0.0007,"performance_1m":-0.0333,"performance_3m":-0.026,"performance_6m":-0.0213,"performance_1y":0.0677,"performance_ytd":0.0093,"analyst_recommendation":2.01,"volume":907130000,"volume_average":462350000,"volume_relative":1.96},{"name":"Consumer Cyclical","performance_1d":0.0021,"performance_1w":0.0004,"performance_1m":-0.1137,"performance_3m":-0.1455,"performance_6m":0.0339,"performance_1y":0.0555,"performance_ytd":-0.095,"analyst_recommendation":1.79,"volume":2750000000,"volume_average":1420000000,"volume_relative":1.94},{"

In [26]:
input1 = "What is the best performing sector for last month"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
agent_executor.invoke({"input": "Which stock are recommented for the best performing sector", "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `get_performance` with `{'group': 'sector'}`


[{"name":"Real Estate","performance_1d":-0.0111,"performance_1w":-0.0027,"performance_1m":-0.0267,"performance_3m":-0.0266,"performance_6m":-0.0765,"performance_1y":0.0618,"performance_ytd":0.0092,"analyst_recommendation":1.98,"volume":660630000,"volume_average":371640000,"volume_relative":1.78},{"name":"Consumer Defensive","performance_1d":-0.0018,"performance_1w":-0.0007,"performance_1m":-0.0333,"performance_3m":-0.026,"performance_6m":-0.0213,"performance_1y":0.0677,"performance_ytd":0.0093,"analyst_recommendation":2.01,"volume":907130000,"volume_average":462350000,"volume_relative":1.96},{"name":"Consumer Cyclical","performance_1d":0.0021,"performance_1w":0.0004,"performance_1m":-0.1137,"performance_3m":-0.1455,"performance_6m":0.0339,"performance_1y":0.0555,"performance_ytd":-0.095,"analyst_recommendation":1.79,"volume":2750000000,"volume_average":1420000000,"volume_relative":1.94},{"

{'input': 'Which stock are recommented for the best performing sector',
 'chat_history': [HumanMessage(content='What is the best performing sector for last month', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The best performing sector for last month was the "Consumer Defensive" sector with a performance of -3.33%.', additional_kwargs={}, response_metadata={})],
 'output': 'Here are some stock recommendations for the best performing sector "Consumer Defensive" based on strong buy recommendations:\n\n1. **Hims & Hers Health Inc (HIMS)**\n   - Industry: Household & Personal Products\n   - Market Cap: $7.72 billion\n   - Price: $34.75\n   - Change Percent: +0.0572\n\n2. **Oatly Group AB ADR (OTLY)**\n   - Industry: Packaged Foods\n   - Market Cap: $299.9 million\n   - Price: $10.02\n   - Change Percent: +0.057\n\n3. **Herbalife Ltd (HLF)**\n   - Industry: Packaged Foods\n   - Market Cap: $853.09 million\n   - Price: $8.43\n   - Change Percent: +0.0551\n\n4. **Celsius 